In [5]:
from pytube import YouTube
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import wave
import io
import csv
import subprocess
import pandas as pd
import time

In [6]:
phrases = pd.read_csv('TextData/phrases.csv')
phrases.head()

,Unnamed: 0,id,context,correctness,growth,text
0,0,1,homework,correct,8,good job answering all the questions
1,1,2,paper,correct,9,you are a natural writer
2,2,3,homework,partial,5,received
3,3,4,paper,incorrect,2,bad writing
4,4,5,homework,correct,8,keep up the good work!


In [7]:
CONTEXTS = phrases['context'].unique().tolist()
print(CONTEXTS)
CORRECTNESS = phrases['correctness'].unique().tolist()
print(CORRECTNESS)


['homework', 'paper', 'general']
['correct', 'partial', 'incorrect']


In [8]:
def get_text(file):
    r = sr.Recognizer()
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    try:
        return r.recognize_google(audio)
    except sr.UnknownValueError as ve:
        print("Warn: unrecognizable phrase")
    except sr.RequestError as re:
        print("Err: request error")

In [9]:
print(get_text("Audio/Test1.wav"))

great work on your paper I really enjoyed your paper good job finishing all the problems


In [10]:
CACHE = {}
#minimum silence duration to detect end of phrase in ms
MIN_SILENCE_LEN=1000
#the upper bound for how quiet is silent in dFBS
SILENCE_THRESH=-32
#silence added to the new audio segment in ms
KEEP_SILENCE=500
#the maximim seconds of audio processed at a time (text-to-speech API cannot handle long audio files)
SPLIT_ON= 50
def get_lines(fileName):
    SPLIT_ON = 10
    print('Converting audio to text...')
    seg = AudioSegment.from_wav(fileName)
    chunks = split_on_silence(seg, min_silence_len=MIN_SILENCE_LEN, silence_thresh=SILENCE_THRESH, keep_silence=KEEP_SILENCE)
    lines = []
    for i, chunk in enumerate(chunks):
        x = io.BytesIO()
        chunk.export(x, format='wav')
        wav = wave.open(x, 'r')
        dur = wav.getnframes() / wav.getframerate()
        text = ''
        if (dur >= SPLIT_ON):
            t = 0
            for j in range(1, int(dur / SPLIT_ON)):
                y = io.BytesIO()
                chunk[t * SPLIT_ON * 1000:j * SPLIT_ON * 1000].export(y, format='wav')
                text += str(get_text(y)) + ' '
                t = j
            y = io.BytesIO()
            chunk[t * SPLIT_ON * 1000:int(dur) * 1000].export(y, format='wav')
            text += str(get_text(y)) + ' '
        else:
            x = io.BytesIO()
            chunk.export(x, format='wav')
            text = str(get_text(x))
        if text != 'None':
            print('Phrase:"'+ text+'"')
            entry = input('Enter any key to read phrase (0 to skip, blank for last entry)?')
            if entry !='0':
                if len(entry) == 0:
                    lines.append({'id':hash(text),'text':text, 'context':CACHE['context'],
                                  'correct':CACHE['correct'],'growth':CACHE['growth']})
                else:
                    context =input('Context ('+str(CONTEXTS)+'):').lower()
                    correct =input('Correctness ('+str(CORRECTNESS)+'):').lower()
                    while correct not in CORRECTNESS:
                        correct=input('Enter a valid correctness value:')
                    growth =input('Growth Mindset (score from 1 to 10):')
                    while int(growth) <1 or int(growth) >10:
                        growth=input('Enter score from 1 to 10:')
                    CACHE['context']=context
                    CACHE['correctness']=correct
                    CACHE['growth']=growth
                    lines.append({'id':hash(text), 'context':context, 'correctness':correct,'growth':growth,'text':text})
    return (lines)

In [14]:
for phrase in get_lines("Audio/Test2.wav"):
    phrases.append(phrase,ignore_index=True)

Converting audio to text...
Phrase:"great work on your paper"
Enter any key to read phrase (0 to skip, blank for last entry)?e
Context (['homework', 'paper', 'general']):paper
Correctness (['correct', 'partial', 'incorrect']):correct
Growth Mindset (score from 1 to 10):7
Phrase:"I did not like your paper"
Enter any key to read phrase (0 to skip, blank for last entry)?e
Context (['homework', 'paper', 'general']):paper
Correctness (['correct', 'partial', 'incorrect']):incorrect
Growth Mindset (score from 1 to 10):1
Phrase:"good job finishing all the problems"
Enter any key to read phrase (0 to skip, blank for last entry)?e
Context (['homework', 'paper', 'general']):homework
Correctness (['correct', 'partial', 'incorrect']):correct
Growth Mindset (score from 1 to 10):7


In [15]:
phrases.to_csv('TextData/phrases.csv')